In [1]:
import pandas as pd
import json
from typing import List
import csv

In [2]:
def preprocessing_raw_data(raw, file_type):
    '''Exports preprocessed raw data file'''
  
    container = []  # List of lists
    with open(raw, encoding = "utf-8") as raw_file:
        in_raw = csv.reader(raw_file, delimiter='\t', quotechar='|')
        for row in in_raw:
            if row:
                if row[0].startswith('#'):
                    continue
                else:
                    container.append(row)
                      
    with open('../data/clean_raw_'+file_type+'_data.tsv', 'w', encoding = "utf-8", newline = "") as output_file:
        writer = csv.writer(output_file, delimiter='\t', quotechar='|')
        # Getting the max line size
        padding_limit = max([len(line) for line in container])
        sentence_nr = []
        line_iter = 0
        for line in container:
            while len(line) != padding_limit:
                line.append('_')
            try:
                if int(line[0]) == 1:  # New line
                    line_iter += 1
            except ValueError:  # Some lines have periods in the index
                line_iter += 1
            writer.writerow([line_iter] + line)


In [3]:
def change_value(preprocessed_data, file_type):

    df = pd.read_csv('../data/clean_raw_'+file_type+'_data.tsv', sep='\t', header=None , encoding = "utf-8",
                     keep_default_na=False, quotechar='|', skip_blank_lines=False)

    length = len(df.iloc[0].tolist())
    for i in range(length-11):
        row = i + 11 
        df.iloc[:, row]= df.iloc[:, row].apply(lambda x: 'O' if x in ['', '_'] else x).tolist()
        if row >=12:
            df.iloc[:, row]= df.iloc[:, row].apply(lambda x: "B-"+x if x 
                                                   not in ['', 'O'] else x).tolist()  
    return df

In [4]:
def from_csv_to_json(df, file_type):
    
    schemas = [] # set up container 

    # group sentence from sentences 
    sentences = df.groupby(0)
    sum_length = 0
    for sentence in sentences:
        flag = 1
        num, df = sentence
        for i in range(len(df)):
            # sentence = sentences.get_group()
            # print(sentence)
            # nt_length = len(sentence[0])
         
          # from each sentence find the index of their verbs
            verbs= df[11].iloc[i]  
            if verbs == "O":
                continue            
            # rbs = sentence[sentence.iloc[:, 11] != 'O']
            try:
                sent_number = df[df.iloc[:, 1]]
    
            except Exception as e:
                print(df.head())
                print(e, "ERROR")
                continue
    
            if flag == 1:
                V = "V"
            
            else:
                V = "_"
            
            flag += 1
            
            # for count, (sent_n, verb) in enumerate(zip(sent_number, verbs)):
            #     print(verb)
            #     # s_n = 1
            #     if count==0:
            #         V = "V"
            #     else:
            #         V = "_"
                # print(verbs)
                # if i!=0 and count ==0:
                #     sum_length+=sent_length-1
                #     print(sum_length)
                #     print(verb)
                # if i!=0:
                #     verb_dice = verb-sum_length-sum_length
                # else:
                #     verb_dice = verb

            schema = {
                    'sep_words': df.iloc[:, 2].tolist(),
                    'BIO': df.iloc[:, 12].tolist(),
                    'pred_sense': [i, df.iloc[i, 11], V, df.iloc[i, 5]]
            }
    
            schemas.append(schema)
          
            # int(schema)
        with open("../data/srl_univprop_en."+file_type+".jsonl", 'w') as f:   

            for entry in schemas:
                json.dump(entry, f)
                f.write('\n')


In [ ]:
file_type = ['train', 'dev']
# if __name__ == '__main__':
for file in file_type:
    path  = "../data/srl_univprop_en."+file+".conll"     
    preprocessed_data = preprocessing_raw_data(path, file)
    df = change_value(preprocessed_data, file)
    from_csv_to_json(df, file)

/tmp/ipykernel_9389/2062762571.py:6: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  df = change_value(preprocessed_data, file)


     0    1        2        3      4      5   \
505  25  1.0        (        (  PUNCT  -LRB-   
506  25  2.0     This     this   PRON     DT   
507  25  3.0       is       be    AUX    VBZ   
508  25  4.0        a        a    DET     DT   
509  25  5.0  largely  largely    ADV     RB   

                                                    6  7       8         9   \
505                                                  _  8   punct   8:punct   
506                           Number=Sing|PronType=Dem  8   nsubj   8:nsubj   
507  Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...  8     cop     8:cop   
508                          Definite=Ind|PronType=Art  8     det     8:det   
509                                                  _  8  advmod  8:advmod   

     ... 37 38 39 40 41 42 43 44 45 46  
505  ...  O  O  O  O  O  O  O  O  O  O  
506  ...  O  O  O  O  O  O  O  O  O  O  
507  ...  O  O  O  O  O  O  O  O  O  O  
508  ...  O  O  O  O  O  O  O  O  O  O  
509  ...  O  O  O  O  O  O  O  

      0    1     2     3      4    5                          6  7       8   \
1037  49  1.0  Even  even    ADV   RB                          _  3  advmod   
1038  49  2.0   the   the    DET   DT  Definite=Def|PronType=Art  3     det   
1039  49  3.0    IP    IP  PROPN  NNP                Number=Sing  7   nsubj   
1040  49  4.0   and   and  CCONJ   CC                          _  5      cc   
1041  49  5.0  ICDC  ICDC  PROPN  NNP                Number=Sing  3    conj   

                      9   ... 37 38 39 40 41 42 43 44 45 46  
1037            3:advmod  ...  O  O  O  O  O  O  O  O  O  O  
1038               3:det  ...  O  O  O  O  O  O  O  O  O  O  
1039             7:nsubj  ...  O  O  O  O  O  O  O  O  O  O  
1040                5:cc  ...  O  O  O  O  O  O  O  O  O  O  
1041  3:conj:and|7:nsubj  ...  O  O  O  O  O  O  O  O  O  O  

[5 rows x 47 columns]
'[47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0] not in index' ERROR
      0    1     2     3

[5 rows x 47 columns]
'[47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0] not in index' ERROR
      0    1          2         3     4    5   \
1248  54  1.0       What      what  PRON   WP   
1249  54  2.0  irritates  irritate  VERB  VBZ   
1250  54  3.0         me         I  PRON  PRP   
1251  54  4.0         is        be   AUX  VBZ   
1252  54  5.0       this      this   DET   DT   

                                                     6  7      8        9   \
1248                                       PronType=Int  2  nsubj  2:nsubj   
1249  Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...  9  csubj  9:csubj   
1250         Case=Acc|Number=Sing|Person=1|PronType=Prs  2    obj    2:obj   
1251  Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...  9    cop    9:cop   
1252                           Number=Sing|PronType=Dem  9    det    9:det   

      ... 37 38 39 40 41 42 43 44 45 46  
1248  ...  O  O  O  O  O

      0    1        2        3      4    5   \
1619  64  1.0       He       he   PRON  PRP   
1620  64  2.0    added      add   VERB  VBD   
1621  64  3.0     that     that  SCONJ   IN   
1622  64  4.0        "        "  PUNCT   ``   
1623  64  5.0  America  America  PROPN  NNP   

                                                     6  7      8        9   \
1619  Case=Nom|Gender=Masc|Number=Sing|Person=3|Pron...  2  nsubj  2:nsubj   
1620                   Mood=Ind|Tense=Past|VerbForm=Fin  0   root   0:root   
1621                                                  _  8   mark   8:mark   
1622                                                  _  8  punct  8:punct   
1623                                        Number=Sing  8  nsubj  8:nsubj   

      ... 37 38 39 40 41 42 43 44 45 46  
1619  ...  O  O  O  O  O  O  O  O  O  O  
1620  ...  O  O  O  O  O  O  O  O  O  O  
1621  ...  O  O  O  O  O  O  O  O  O  O  
1622  ...  O  O  O  O  O  O  O  O  O  O  
1623  ...  O  O  O  O  O  O  O  O  O  

...

         0    1          2          3      4    5   \
28837  1338  1.0         If         if  SCONJ   IN   
28838  1338  2.0         by         by    ADP   IN   
28839  1338  3.0  democracy  democracy   NOUN   NN   
28840  1338  4.0         we         we   PRON  PRP   
28841  1338  5.0       mean       mean   VERB  VBP   

                                               6   7      8            9   \
28837                                           _   5   mark       5:mark   
28838                                           _   3   case       3:case   
28839                                 Number=Sing   5    obl     5:obl:by   
28840  Case=Nom|Number=Plur|Person=1|PronType=Prs   5  nsubj      5:nsubj   
28841            Mood=Ind|Tense=Pres|VerbForm=Fin  67  advcl  67:advcl:if   

       ... 37 38 39 40 41 42 43 44 45 46  
28837  ...  O  O  O  O  O  O  O  O  O  O  
28838  ...  O  O  O  O  O  O  O  O  O  O  
28839  ...  O  O  O  O  O  O  O  O  O  O  
28840  ...  O  O  O  O  O  O  O  O  O  O

         0    1      2      3      4   5                          6   7   \
28966  1342  1.0     On     on    ADP  IN                          _   4   
28967  1342  2.0    the    the    DET  DT  Definite=Def|PronType=Art   4   
28968  1342  3.0  other  other    ADJ  JJ                 Degree=Pos   4   
28969  1342  4.0   hand   hand   NOUN  NN                Number=Sing  20   
28970  1342  5.0      ,      ,  PUNCT   ,                          _  20   

          8          9   ... 37 38 39 40 41 42 43 44 45 46  
28966   case     4:case  ...  O  O  O  O  O  O  O  O  O  O  
28967    det      4:det  ...  O  O  O  O  O  O  O  O  O  O  
28968   amod     4:amod  ...  O  O  O  O  O  O  O  O  O  O  
28969    obl  20:obl:on  ...  O  O  O  O  O  O  O  O  O  O  
28970  punct   20:punct  ...  O  O  O  O  O  O  O  O  O  O  

[5 rows x 47 columns]
'[47.0, 48.0, 49.0, 50.0, 51.0, 52.0] not in index' ERROR
         0    1      2      3      4   5                          6   7   \
28966  1342  1.0    

         0    1       2       3      4    5             6  7       8   \
29461  1360  1.0      On      on    ADP   IN             _  2    case   
29462  1360  2.0  August  August  PROPN  NNP   Number=Sing  9     obl   
29463  1360  3.0       9       9    NUM   CD  NumType=Card  2  nummod   
29464  1360  4.0       ,       ,  PUNCT    ,             _  2   punct   
29465  1360  5.0    2004    2004    NUM   CD  NumType=Card  2  nummod   

             9   ... 37 38 39 40 41 42 43 44 45 46  
29461    2:case  ...  O  O  O  O  O  O  O  O  O  O  
29462  9:obl:on  ...  O  O  O  O  O  O  O  O  O  O  
29463  2:nummod  ...  O  O  O  O  O  O  O  O  O  O  
29464   2:punct  ...  O  O  O  O  O  O  O  O  O  O  
29465  2:nummod  ...  O  O  O  O  O  O  O  O  O  O  

[5 rows x 47 columns]
'[47.0, 48.0] not in index' ERROR
         0    1       2       3      4    5             6  7       8   \
29461  1360  1.0      On      on    ADP   IN             _  2    case   
29462  1360  2.0  August  August  PROPN 

         0    1         2        3      4    5   \
30392  1393  1.0       The      the    DET   DT   
30393  1393  2.0      Wall     Wall  PROPN  NNP   
30394  1393  3.0    Street   Street  PROPN  NNP   
30395  1393  4.0   Journal  Journal  PROPN  NNP   
30396  1393  5.0  reported   report   VERB  VBD   

                                     6  7         8           9   ... 37 38  \
30392         Definite=Def|PronType=Art  4       det       4:det  ...  O  O   
30393                       Number=Sing  3  compound  3:compound  ...  O  O   
30394                       Number=Sing  4  compound  4:compound  ...  O  O   
30395                       Number=Sing  5     nsubj     5:nsubj  ...  O  O   
30396  Mood=Ind|Tense=Past|VerbForm=Fin  0      root      0:root  ...  O  O   

      39 40 41 42 43 44 45 46  
30392  O  O  O  O  O  O  O  O  
30393  O  O  O  O  O  O  O  O  
30394  O  O  O  O  O  O  O  O  
30395  O  O  O  O  O  O  O  O  
30396  O  O  O  O  O  O  O  O  

[5 rows x 47 columns]
'[4

         0    1      2      3      4    5             6   7          8   \
31252  1424  1.0     In     in    ADP   IN             _   2       case   
31253  1424  2.0   Fall   fall   NOUN   NN   Number=Sing  14        obl   
31254  1424  3.0   2004   2004    NUM   CD  NumType=Card   2  nmod:tmod   
31255  1424  4.0      ,      ,  PUNCT    ,             _  14      punct   
31256  1424  5.0  MSNBC  MSNBC  PROPN  NNP   Number=Sing  14      nsubj   

                9   ... 37 38 39 40 41 42 43 44 45 46  
31252       2:case  ...  O  O  O  O  O  O  O  O  O  O  
31253    14:obl:in  ...  O  O  O  O  O  O  O  O  O  O  
31254  2:nmod:tmod  ...  O  O  O  O  O  O  O  O  O  O  
31255     14:punct  ...  O  O  O  O  O  O  O  O  O  O  
31256     14:nsubj  ...  O  O  O  O  O  O  O  O  O  O  

[5 rows x 47 columns]
'[47.0] not in index' ERROR
         0    1      2      3      4    5             6   7          8   \
31252  1424  1.0     In     in    ADP   IN             _   2       case   
31253  1424 